In [1]:
import torch
import torch.nn as nn
import timm
from timm.models.resnet import * 


/h/abbasgln/.conda/envs/tta/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
timm.list_models('resnet*')
model = timm.create_model(
    'resnet14t',
    num_classes=2,
    in_chans=1,
    features_only=False,
    norm_layer= lambda channels: nn.GroupNorm(num_groups=8, num_channels=channels)
    )
isinstance(model, nn.Module)
model

ResNet(
  (conv1): Sequential(
    (0): Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): GroupNorm(8, 24, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(24, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): GroupNorm(8, 32, eps=1e-05, affine=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): GroupNorm(8, 64, eps=1e-05, affine=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): GroupNorm(8, 64, eps=1e-05, affine=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): GroupNorm(8, 64, eps=1e-05, affine=True)
      (drop_block): Identi

In [3]:
model.feature_info[-1]

{'num_chs': 2048, 'reduction': 32, 'module': 'layer4'}

In [4]:
from timm.layers import create_classifier 
global_pool, fc = create_classifier(model.feature_info[-1]['num_chs'], 2, pool_type='avg')

In [5]:
global_pool, fc

(SelectAdaptivePool2d(pool_type=avg, flatten=Flatten(start_dim=1, end_dim=-1)),
 Linear(in_features=2048, out_features=2, bias=True))

In [6]:
import numpy as np
from dotenv import load_dotenv
import os
import pandas as pd

load_dotenv()

DATA_ROOT = os.environ.get('DATA_ROOT')

cores_path = os.path.join(DATA_ROOT, 'cores_dataset')
cores_path

'/ssd005/projects/exactvu_pca/cores_dataset'

In [7]:
cores_id_to_names = pd.DataFrame(np.load(os.path.join(cores_path, 'core_specifiers.npy')))
cores_id_to_names

,0
0,CRCEO-0004_LML
1,CRCEO-0004_RBL
2,CRCEO-0005_LML
3,CRCEO-0005_LMM
4,CRCEO-0005_RBL
...,...
6533,UVA-0628_RAM
6534,UVA-0628_RBL
6535,UVA-0628_RBM
6536,UVA-0628_RML


In [8]:
cores_id_to_names.iloc[0, 0]


'CRCEO-0004_LML'

In [9]:
core_csv_path = os.path.join(cores_path, 'core.csv')

df = pd.read_csv(core_csv_path)
print(df.head())


   id  loc   grade  pct_cancer  primary_grade  secondary_grade  \
0   0  LAM  Benign         NaN            NaN              NaN   
1   1  LBM  Benign         NaN            NaN              NaN   
2   2  LML  Benign         NaN            NaN              NaN   
3   3  LMM  Benign         NaN            NaN              NaN   
4   4  RBL  Benign         NaN            NaN              NaN   

              tag  patient_id  
0  CRCEO-0004_LAM           0  
1  CRCEO-0004_LBM           0  
2  CRCEO-0004_LML           0  
3  CRCEO-0004_LMM           0  
4  CRCEO-0004_RBL           0  


In [10]:
pd.DataFrame(np.load(os.path.join(DATA_ROOT, 'bmode_learning_data/nct', 'needle_mask.npy')))

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10043,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10044,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10045,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10046,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
%cd ~/codes/medAI
%ls
from projects.tta.datasets.datasets import *


/fs01/home/abbasgln/codes/medAI
LICENSE  medAI/  projects/  README.md  setup.py


/h/abbasgln/.conda/envs/tta/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/h/abbasgln/.conda/envs/tta/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [12]:
cohort_selection_options = CohortSelectionOptions(
    fold = 0,
    n_folds = 5,
    min_involvement = .4,
    remove_benign_from_positive_patients = True,
    benign_to_cancer_ratio = .5,
    seed = 0,
    )


patch_options = PatchOptions(
    patch_size_mm = (5, 5),
    strides = (1, 1),  # defines the stride in mm of the base positions
    needle_mask_threshold = -1,  # if not None, then only positions with a needle mask intersection greater than this value are kept
    prostate_mask_threshold = 0.5,
    shift_delta_mm = 0.0, # whether to randomly shift the patch by a small amount
    )  

# rf_dataset = ExactNCT2013RFImages(split='train', cohort_selection_options=cohort_selection_options)
# rf_patch_dataset = ExactNCT2013RFPatches(split='test', cohort_selection_options=cohort_selection_options, patch_options=patch_options)
# len(rf_patch_dataset)

In [13]:
# import matplotlib.pyplot as plt
# plt.imshow(rf_patch_dataset[0]['patch'])
# rf_patch_dataset[124234]

## Dataset with support patches

In [1]:
import torch
import numpy as np
from dotenv import load_dotenv
import os
import pandas as pd

from torchvision.transforms import InterpolationMode
from torchvision.transforms import v2 as T, Compose
from torchvision.tv_tensors import Image as TVImage

load_dotenv()

DATA_ROOT = os.environ.get('DATA_ROOT')

%cd ~/codes/medAI
%ls
from projects.tta.datasets.datasets import *

/fs01/home/abbasgln/codes/medAI
build/           LICENSE  medAI/           projects/               README.md
first_test_exp/  logs/    medAI.egg-info/  projects_tta.egg-info/  setup.py


/h/abbasgln/.conda/envs/mttt/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/h/abbasgln/.conda/envs/mttt/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
class Transform:
    def __init__(selfT, augment=False):
        selfT.augment = augment
        selfT.size = (256, 256)
    
    def __call__(selfT, item):
        patch = item.pop("patch")
        patch = (patch - patch.min()) / (patch.max() - patch.min())
        patch = TVImage(patch)
        patch = T.ToTensor()(patch)
        patch = T.Resize(selfT.size, antialias=True)(patch)

        support_patches = item.pop("support_patches")
        # Normalize support patches along last two dimensions
        support_patches = (support_patches - support_patches.min(axis=(1, 2), keepdims=True)) \
        / (support_patches.max(axis=(1,2), keepdims=True) \
            - support_patches.min(axis=(1, 2), keepdims=True))
        support_patches = TVImage(support_patches)
        support_patches = T.ToTensor()(support_patches)
        support_patches = T.Resize(selfT.size, antialias=True)(support_patches)
        
        # Augment support patches
        transform = T.Compose([
            T.RandomAffine(degrees=0, translate=(0.1, 0.1)),
            T.RandomHorizontalFlip(p=0.5),
            T.RandomVerticalFlip(p=0.5),
        ])   
        support_patches_aug1, support_patches_aug2 = transform(support_patches, support_patches)
        
        label = torch.tensor(item["grade"] != "Benign").long()
        return support_patches_aug1, support_patches_aug2, patch, label, item


In [ ]:

cohort_selection_options = CohortSelectionOptions(
    fold = 0,
    n_folds = 5,
    min_involvement = 40,
    remove_benign_from_positive_patients = True,
    benign_to_cancer_ratio = 1,
    seed = 0,
    )


patch_options = PatchOptions(
    patch_size_mm = (5, 5),
    strides = (1, 1),  # defines the stride in mm of the base positions
    needle_mask_threshold = 0.5,  # if not None, then only positions with a needle mask intersection greater than this value are kept
    prostate_mask_threshold = 0.5,
    shift_delta_mm = 0.0, # whether to randomly shift the patch by a small amount
    )  

rf_patch_dataset_with_support = ExactNCT2013RFPatchesWithSupportPatches(
    split='train',
    transform=Transform(),
    cohort_selection_options=cohort_selection_options,
    patch_options=patch_options,
    debug=False
    )
len(rf_patch_dataset_with_support)


In [4]:
rf_patch_dataset_with_support[55][-1]

/h/abbasgln/.conda/envs/pytorch210/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


{'patient_id': 4,
 'id': 46,
 'loc': 'RAL',
 'grade': 'Benign',
 'pct_cancer': nan,
 'primary_grade': nan,
 'secondary_grade': nan,
 'tag': 'CRCEO-0009_RAL',
 'case_id': 'CRCEO-0009',
 'case_number': 9,
 'center': 'CRCEO',
 'age': 78,
 'family_history': False,
 'psa': 5.4,
 'label': False,
 'position': array([ 9., 31., 14., 36.])}

In [4]:
import torch
torch.cuda.is_available()

True

In [9]:
from datasets.datasets import ExactNCT2013RFPatches, CohortSelectionOptions, SupportPatchConfig

class Transform:
    def __init__(selfT, augment=False):
        selfT.augment = augment
        selfT.size = (256, 256)
    
    def __call__(selfT, item):
        patch = item.pop("patch")
        patch = (patch - patch.min()) / (patch.max() - patch.min())
        patch = TVImage(patch)
        patch = T.ToTensor()(patch)
        patch = T.Resize(selfT.size, antialias=True)(patch) 
        
        if selfT.augment:
            # Augment support patches
            transform = T.Compose([
                T.RandomAffine(degrees=0, translate=(0.1, 0.1)),
                T.RandomHorizontalFlip(p=0.5),
                T.RandomVerticalFlip(p=0.5),
            ])  
            patch = transform(patch)
        
        label = torch.tensor(item["grade"] != "Benign").long()
        return patch, label, item

train_ds = ExactNCT2013RFPatches(
    split="train",
    transform=Transform(augment=False),
    cohort_selection_options=CohortSelectionOptions(
        benign_to_cancer_ratio=1,
        min_involvement=40,
        remove_benign_from_positive_patients=True,
        fold=0,
    ),
    debug=True,
)


from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_ds, batch_size=32, shuffle=True, num_workers=4
)

Computing positions train:  15%|█▌        | 101/672 [00:01<00:05, 98.03it/s]


In [ ]:
for i in tqdm(train_loader):
    i[0].cuda()
    i[1].cuda()